# 🧠 OpenAI Vector Store Setup
Dieses Notebook lädt eine Datei hoch, erstellt einen Vector Store, verbindet ihn mit einem Assistant und ermöglicht die semantische Suche mit GPT-4.

In [ ]:
# 📦 Installiere notwendige Pakete
!pip install openai python-dotenv

In [ ]:
# 🔐 API-Key einbinden (über .env Datei empfohlen)
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# 📁 Datei hochladen
from pathlib import Path

file_path = "data/dein_dokument.pdf"  # Passe dies an deine Datei an
file = openai.files.create(file=open(file_path, "rb"), purpose="assistants")
print("✅ Datei hochgeladen:", file.id)

In [ ]:
# 🧠 Vector Store erstellen
vector_store = openai.beta.vector_stores.create(name="Mein Vector Store")
print("📌 Vector Store ID:", vector_store.id)

In [ ]:
# ➕ Datei zum Vector Store hinzufügen
openai.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=[file.id]
)
print("✅ Datei im Vector Store gespeichert")

In [ ]:
# 🤖 Assistant erstellen
assistant = openai.beta.assistants.create(
    name="Dokumenten-Helfer",
    instructions="Beantworte Fragen basierend auf dem Dokument.",
    tools=[{"type": "retrieval"}],
    model="gpt-4",
    vector_store_ids=[vector_store.id],
)
print("🧠 Assistant-ID:", assistant.id)

In [ ]:
# 💬 Thread starten
thread = openai.beta.threads.create()
print("💬 Thread-ID:", thread.id)

In [ ]:
# ❓ Frage stellen und Antwort erhalten
def frage_stellen(frage: str):
    openai.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=frage,
    )
    run = openai.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    messages = openai.beta.threads.messages.list(thread_id=thread.id)
    for msg in reversed(messages.data):
        if msg.role == "assistant":
            return msg.content[0].text.value
    return "Keine Antwort gefunden."

# Beispiel-Frage:
frage_stellen("Was steht in Kapitel 3?")


👉 Jetzt kannst du oben deine Datei anpassen und beliebige Fragen stellen.